In [ ]:
#| default_exp datamodule

# datamodule

> API details.

In [ ]:
#|hide
from nbprocess.showdoc import *

In [ ]:
#|export
from fastcore.all import *
import pandas as pd
import numpy as np

In [ ]:
fpath = 'https://raw.githubusercontent.com/Isaac-Flath/trading/main/data/eod-quotemedia.csv'


## Getters

In [ ]:
#|export
class CsvGetter:
    def __init__(self,fpath): store_attr()
    def __call__(self): return pd.read_csv(self.fpath, parse_dates=['date'])

In [ ]:
df = CsvGetter(fpath)()
test_eq(df.shape,(490737,3))

## Setters

In [ ]:
#|export
class SizeSplitter:
    def __init__(self,sizes=None): 
        if sizes is None:
            self.sizes = L(('train',0.5),('valid',0.25),('test',0.25))
        else: 
            sizes_sum = np.array([o[1] for o in sizes]).sum()
            
            try:test_close(sizes_sum, 1.)
            except AssertionError: raise Exception('Your sizes must sum to 1')
                
            try:test_eq(['train','valid','test'],sizes.keys())
            except AssertionError: raise Exception('You must have train, valid, and test sets')
                
            self.sizes = sizes

        
    def __call__(self,df):        
        sizes = self.sizes
        unique_dates = L(*df.date.unique()).sorted()

        out = AttrDict()
        
        break_sz = int(len(unique_dates)*sizes[0][1])
        out[sizes[0][0]] = df.loc[df.date <= unique_dates[break_sz]]
        
        _remainder_df = df.loc[df.date > unique_dates[break_sz]]
        _remainder_dates = unique_dates[break_sz+1:]

        break_sz = int(len(unique_dates)*sizes[1][1])
        out[sizes[1][0]] = _remainder_df.loc[_remainder_df.date <= _remainder_dates[break_sz]]
        out[sizes[2][0]] = _remainder_df.loc[_remainder_df.date > _remainder_dates[break_sz]]
        return out

In [ ]:
df = CsvGetter(fpath)()
dataset = SizeSplitter()(df)
test_eq(dataset.train.shape,(242680, 3))
test_eq(dataset.keys(),['train','valid','test'])

## DataModule

In [ ]:
#| export
class DataModule:
    def __init__(self, getter, splitter): 
        store_attr()
        df = getter()
        self.datasets = splitter(df)

In [ ]:
dm = DataModule(CsvGetter(fpath),SizeSplitter())
test_eq(dm.datasets.train.shape,(242680, 3))
test_eq(dm.datasets.keys(),['train','valid','test'])